In [140]:
## import modules
import json
import re
import pandas as pd
import nltk
import math
from math import sqrt
import sklearn
import statsmodels.api as sm
from nltk.stem import WordNetLemmatizer 
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
import numpy as np
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
from tqdm import tqdm
from sklearn import linear_model
import collections
from gensim.models import Word2Vec
from gensim.models import word2vec
import multiprocessing
import time
wnl = WordNetLemmatizer() 
sr = stopwords.words('english')
table = str.maketrans('', '', string.punctuation)

In [141]:
## read data
## define try size 
trysize=5000
file_name = 'data/review_train.json'
data_train = []
size=0
with open(file_name, 'r') as f:
    try:
        while size<=(trysize-1):
            line = f.readline()
            size=size+1
            if line:
                r = json.loads(line)
                data_train.append(r)
            else:
                break
    except:
        f.close()

In [142]:
## data processiong
review_train_clean = [[1]]*trysize
##a list  with dict element,you can get a summary of frequency of one review by [i] and frequency for a specific word by [i]['word']
frequency_train_clean = [[1]]*trysize  
for i in tqdm(range(trysize)):
    #Split into words
    x = word_tokenize(review_train[i]['text'])
    #Convert to lower case
    x = [w.lower() for w in x]
    #Remove punctuation
    x = [w.translate(table) for w in x]
    #Remove not alphabetic
    x = [word for word in x if word.isalpha()]
    #Remove stop words
    x = [w for w in x if not w in sr]
    ## lemmatization
    x = [wnl.lemmatize(w) for w in x]
    review_train_clean[i] = x
    frequency_train_clean[i] = collections.Counter(x)

100%|██████████| 5000/5000 [00:06<00:00, 765.08it/s]


In [129]:
## train words embedding model
## define dimension of need
size = 300
start = time.process_time()
w2v = Word2Vec(review_train_clean, size=size, window=10, min_count=1,
            workers=multiprocessing.cpu_count(), sg=1, iter=10, negative=20)
end = time.process_time()
print('Running time: %s Seconds'%(end-start))
## save model as w2vmodel
w2v.save('w2vmodel')
## load model
model = Word2Vec.load('w2vmodel')

Running time: 825.3150559999999 Seconds


In [137]:
## for each review, get 1*size matrix use average of words matrix 
trysize=5000
predictors = np.empty((trysize,size), float)
for i in tqdm(range(trysize)):
    predictors[i,] = pd.DataFrame(model.wv[review_train_clean[i]]).mean()
    
response = pd.DataFrame(data_train)['stars']


100%|██████████| 5000/5000 [00:01<00:00, 2687.83it/s]


In [155]:
# for each review, we compute TF_IDF value for all word, and pick the first five,
# then combine all first fives and pick first 100 as our predictors for logistic regression.

##compute how many reviews in which this key appears
def f_appear_num(word,dat):
    num=0
    for i in range(0,len(dat)):
        d = dat[i]['text']
        if word in d == True:
            num = num+1
    return num
def tfidf_4_web(revw,dat):
    ## input is a list of words for one review
    text = {}
    for key in revw:
        text[key] = text.get(key, 0) + 1
    tf = dict()
    idf = dict()
    tfidfvalue = dict()
    for key in text:
        num = f_appear_num(key,dat)
        tf[key] = text[key]/len(text)
        idf[key] = math.log(len(dat)/(1+num)) 
        tfidfvalue[key]= tf[key]*idf[key]
    return tfidfvalue

In [180]:
size = 300
trysize=5000
predictors_weighted = np.empty((trysize,size), float)
for i in tqdm(range(trysize)):
    tfidf_dict = tfidf_4_web(review_train_clean[i],data_train)
    tfidf_mul = []
    for word in review_train_clean[i]:
        tfidf_mul.append(tfidf_dict[word])
    predictors_weighted[i,] = (pd.DataFrame(model.wv[review_train_clean[i]]).mul(tfidf_mul,axis=0)).mean()
    
response = pd.DataFrame(data_train)['stars']


100%|██████████| 5000/5000 [11:18<00:00,  9.35it/s]


In [182]:
## logistic regression
model_logit = LogisticRegression(solver = 'lbfgs', multi_class='multinomial',max_iter=500)
model_logit.fit(predictors_weighted, response)
## rmse for regression
y_fitted = model_logit.predict(predictors_weighted)
print("RMSE = ", sqrt(sum(np.square(response-y_fitted)) / len(y_fitted)))

RMSE =  1.331465358167459
